In [14]:
#install.packages("randomForest")
suppressPackageStartupMessages(library(randomForest))

In [15]:
#install.packages("ranger")
suppressPackageStartupMessages(library(ranger))


In [16]:
#install.packages("lifecycle")
suppressPackageStartupMessages(library(lifecycle))

ERROR: Error in library(lifecycle): there is no package called 'lifecycle'


In [17]:
#install.packages("dplyr")
suppressPackageStartupMessages(library(dplyr))

Warning message:
"package 'dplyr' was built under R version 3.6.3"

ERROR: Error: package or namespace load failed for 'dplyr' in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 there is no package called 'lifecycle'


In [18]:
#install.packages("magrittr")
suppressPackageStartupMessages(library(magrittr))

In [19]:
#install.packages("lifecycle")
suppressPackageStartupMessages(library(lifecycle))

ERROR: Error in library(lifecycle): there is no package called 'lifecycle'


In [20]:
#install.packages("tidyverse")
suppressPackageStartupMessages(library(tidyverse))

ERROR: Error in library(tidyverse): there is no package called 'tidyverse'


In [21]:
#install.packages("plyr")
suppressPackageStartupMessages(library(plyr))

In [27]:
input_dir <- "C:/Users/MONSTER/Desktop/TEZ/CCLE/INPUT"
output_dir <- "C:/Users/MONSTER/Desktop/TEZ/CCLE/OUTPUT"

In [28]:
setwd(paste0(output_dir))

In [29]:
Boruta_Histologies <- list.dirs(full.names = FALSE, recursive = FALSE)

In [30]:
outer_list <- list()

In [31]:
set.seed(1234)
for(histology in Boruta_Histologies){ 
  setwd(paste0(output_dir,"/",histology))
  Drugs <- list.dirs(full.names = FALSE, recursive = FALSE)
  
    cat(paste0("\nRandom Forest is running for ", histology))
    flush.console()
  
  List_result_table_IC50_Train <- list()
  List_result_table_IC50_Test <- list()
  inner_list <- list()
  for(drug in Drugs){
    setwd(paste0(output_dir,"/",histology,"/",drug))
      
    cat(paste0("\nRandom Forest is running for ", drug))
    flush.console()
      
    Drug_data <- read.delim(paste(drug, "- Expression Data (IC50) - Normalized.tsv", sep = " "))
    
    
      if(length(Drug_data$Sample_Name) >= 20){ 
      
    Drug_data_without_sample_name <- subset(Drug_data, select = - Sample_Name)
    
    
          
    ind <- sample(2, nrow(Drug_data_without_sample_name), replace=TRUE, prob=c(0.7, 0.3))
    train_Drug_data <- Drug_data_without_sample_name[ind==1,]
    test_Drug_data <- Drug_data_without_sample_name[ind==2,]
    
    n_features <- length(setdiff(names(train_Drug_data), "IC50"))
      
    if(length(setdiff(names(train_Drug_data), "IC50")) >= 5) {
    
    
        
    Drug_rf1 <- ranger(
      IC50 ~ ., 
      data = train_Drug_data,
      mtry = floor(n_features / 3),
      respect.unordered.factors = "order",
      seed = 123
    )
    
    
    (default_rmse <- sqrt(Drug_rf1$prediction.error))
    
    hyper_grid <- expand.grid(
      mtry = floor(n_features * c(.05, .15, .25, .333, .4)),
      min.node.size = c(1, 3, 5, 10), 
      replace = c(TRUE, FALSE),                               
      sample.fraction = c(.5, .63, .8),                       
      rmse = NA                                               
    )
    
         
    for(i in seq_len(nrow(hyper_grid))) {
        

        
      fit <- ranger(
        formula         = IC50 ~ ., 
        data            = train_Drug_data, 
        num.trees       = n_features * 10,
        mtry            = hyper_grid$mtry[i],
        min.node.size   = hyper_grid$min.node.size[i],
        replace         = hyper_grid$replace[i],
        sample.fraction = hyper_grid$sample.fraction[i],
        verbose         = FALSE,
        seed            = 123,
        respect.unordered.factors = 'order',
      )
      
      
      hyper_grid$rmse[i] <- sqrt(fit$prediction.error)
    }
    
    
    rmse <- hyper_grid$rmse[i]
    
    
    hyper_grid %>%
    arrange(rmse) %>%
    mutate(perc_gain = (default_rmse - rmse) / default_rmse * 100)
      
     hyper_grid_new <- hyper_grid[order(hyper_grid$rmse, decreasing = FALSE),]
     best_model_from_rf <- hyper_grid_new[1,]
     
     
      
     fit_best_model_rf <- ranger(
       formula         = IC50 ~ .,
       data            = train_Drug_data,
       num.trees       = n_features * 10,
       mtry            = best_model_from_rf[1,1],
       min.node.size   = best_model_from_rf[1,2],
       replace         = best_model_from_rf[1,3],
       sample.fraction = best_model_from_rf[1,4],
       verbose         = FALSE,
       seed            = 123,
       respect.unordered.factors = 'order',
     )
    
     
     train_prediction <- predict(fit_best_model_rf, train_Drug_data)
     R_Square_Train <- cor(train_Drug_data$IC50, train_prediction$prediction)^2
     
     Pearson_Correlation_Train <- cor(train_Drug_data$IC50, train_prediction$prediction)
     Pearson_Correlation_P_Value_Train <- cor.test(train_Drug_data$IC50, train_prediction$prediction)$p.value
   
     Spearman_Correlation_Train <- cor(train_Drug_data$IC50, train_prediction$prediction, method = "spearman")
     Spearman_Correlation_P_Value_Train <- cor.test(train_Drug_data$IC50, train_prediction$prediction, method = "spearman")$p.value
        
        Square_for_train_RMSE <- (train_prediction$prediction - train_Drug_data$IC50)^2
        Mean_Square_for_train_RMSE <- mean(Square_for_train_RMSE)
     RMSE_Train <- sqrt(Mean_Square_for_train_RMSE)
     
     IC50_Train_and_Prediction <- cbind(train_Drug_data$IC50, train_prediction$prediction)
     colnames(IC50_Train_and_Prediction) <- c("Actual_IC50_Train_Data","Predicted_IC50_Test_Data")
     
     
     
     test_prediction <- predict(fit_best_model_rf, test_Drug_data)
     R_Square_Test <- cor(test_Drug_data$IC50, test_prediction$prediction)^2
     
     Pearson_Correlation_Test <- cor(test_Drug_data$IC50, test_prediction$prediction)
     Pearson_Correlation_P_Value_Test <- cor.test(test_Drug_data$IC50, test_prediction$prediction)$p.value
     Spearman_Correlation_Test <- cor(test_Drug_data$IC50, test_prediction$prediction, method = "spearman")
     Spearman_Correlation_P_Value_Test <- cor.test(test_Drug_data$IC50, test_prediction$prediction, method = "spearman")$p.value
     
        Square_for_test_RMSE <- (test_prediction$prediction - test_Drug_data$IC50)^2
        Mean_Square_for_test_RMSE <- mean(Square_for_test_RMSE)
     RMSE_Test <- sqrt(Mean_Square_for_test_RMSE)
     
     IC50_Test_and_Prediction <- cbind(test_Drug_data$IC50, test_prediction$prediction)
     colnames(IC50_Test_and_Prediction) <- c("Actual_IC50_Test_Data","Predicted_IC50_Test_Data")
     
     
     
     Result_table_drug <- data.frame(
                  Tissue = histology,
                  Drug = drug,
              
                  R_Square_Train,
                  R_Square_Test,
                  
                  Pearson_Correlation_Train,
                  Pearson_Correlation_Test,
                  
                  Pearson_Correlation_P_Value_Train,
                  Pearson_Correlation_P_Value_Test,
                  
                  Spearman_Correlation_Train,
                  Spearman_Correlation_Test,
                  
                  Spearman_Correlation_P_Value_Train,
                  Spearman_Correlation_P_Value_Test,
                  
                  RMSE_Train,
                  RMSE_Test
              
     )
     
     
    
     inner_list[[drug]] <- Result_table_drug
     List_result_table_IC50_Train[[drug]] <- IC50_Train_and_Prediction
     List_result_table_IC50_Test[[drug]] <- IC50_Test_and_Prediction
     
     
     dir.create(paste0(output_dir,"/",histology,"/",drug,"/","Random Forest"))
     setwd(paste0(output_dir,"/",histology,"/",drug,"/","Random Forest"))  
     
     
    
     
     write.table(inner_list[[drug]],
                 paste(drug, "- Result_RF.tsv" , sep = " "),
                 row.names = FALSE,
                 quote = FALSE,
                 sep = "\t")
     
     write.table(List_result_table_IC50_Train[[drug]],
                 paste(drug, "- IC50_Train_Result.tsv" , sep = " "),
                 row.names = FALSE,
                 quote = FALSE,
                 sep = "\t")
     
     write.table(List_result_table_IC50_Test[[drug]],
                 paste(drug, "- IC50_Test_Result.tsv" , sep = " "),
                 row.names = FALSE,
                 quote = FALSE,
                 sep = "\t")
     
        }
  
      }
      
      }
 
  
   
   outer_list[[histology]] <- ldply(inner_list, "rbind")
   
    
    }

    


Random Forest is running for carcinoid-endocrine_tumour
Random Forest is running for carcinoma
Random Forest is running for 17-AAG
Random Forest is running for AEW541
Random Forest is running for AZD0530
Random Forest is running for AZD6244
Random Forest is running for Erlotinib
Random Forest is running for Irinotecan
Random Forest is running for L-685458
Random Forest is running for Lapatinib
Random Forest is running for LBW242
Random Forest is running for Nilotinib
Random Forest is running for Nutlin-3
Random Forest is running for Paclitaxel

Warning message in cor.test.default(train_Drug_data$IC50, train_prediction$prediction, :
"Cannot compute exact p-value with ties"Warning message in cor.test.default(test_Drug_data$IC50, test_prediction$prediction, :
"Cannot compute exact p-value with ties"


Random Forest is running for Panobinostat
Random Forest is running for PD-0325901
Random Forest is running for PD-0332991

Warning message in cor(train_Drug_data$IC50, train_prediction$prediction):
"the standard deviation is zero"Warning message in cor(train_Drug_data$IC50, train_prediction$prediction):
"the standard deviation is zero"Warning message in cor(x, y):
"the standard deviation is zero"Warning message in cor(train_Drug_data$IC50, train_prediction$prediction, method = "spearman"):
"the standard deviation is zero"Warning message in cor(rank(x), rank(y)):
"the standard deviation is zero"Warning message in cor(test_Drug_data$IC50, test_prediction$prediction):
"the standard deviation is zero"Warning message in cor(test_Drug_data$IC50, test_prediction$prediction):
"the standard deviation is zero"Warning message in cor(x, y):
"the standard deviation is zero"Warning message in cor(test_Drug_data$IC50, test_prediction$prediction, method = "spearman"):
"the standard deviation is zero"Warning message in cor(rank(x), rank(y)):
"the standard deviation is zero"


Random Forest is running for PF2341066
Random Forest is running for PHA-665752
Random Forest is running for PLX4720
Random Forest is running for RAF265
Random Forest is running for Sorafenib
Random Forest is running for TAE684
Random Forest is running for TKI258
Random Forest is running for Topotecan
Random Forest is running for ZD-6474
Random Forest is running for chondrosarcoma
Random Forest is running for Ewings_sarcoma-peripheral_primitive_neuroectodermal_tumour
Random Forest is running for fibrosarcoma
Random Forest is running for glioma
Random Forest is running for 17-AAG
Random Forest is running for AEW541
Random Forest is running for AZD0530
Random Forest is running for AZD6244
Random Forest is running for Erlotinib
Random Forest is running for Irinotecan
Random Forest is running for L-685458
Random Forest is running for Lapatinib
Random Forest is running for LBW242
Random Forest is running for Nilotinib
Random Forest is running for Nutlin-3
Random Forest is running for Paclit

Warning message in cor(train_Drug_data$IC50, train_prediction$prediction):
"the standard deviation is zero"Warning message in cor(train_Drug_data$IC50, train_prediction$prediction):
"the standard deviation is zero"Warning message in cor(x, y):
"the standard deviation is zero"Warning message in cor(train_Drug_data$IC50, train_prediction$prediction, method = "spearman"):
"the standard deviation is zero"Warning message in cor(rank(x), rank(y)):
"the standard deviation is zero"Warning message in cor(test_Drug_data$IC50, test_prediction$prediction):
"the standard deviation is zero"Warning message in cor(test_Drug_data$IC50, test_prediction$prediction):
"the standard deviation is zero"Warning message in cor(x, y):
"the standard deviation is zero"Warning message in cor(test_Drug_data$IC50, test_prediction$prediction, method = "spearman"):
"the standard deviation is zero"Warning message in cor(rank(x), rank(y)):
"the standard deviation is zero"


Random Forest is running for AEW541
Random Forest is running for AZD0530
Random Forest is running for AZD6244
Random Forest is running for Erlotinib
Random Forest is running for Irinotecan
Random Forest is running for L-685458
Random Forest is running for Lapatinib
Random Forest is running for LBW242
Random Forest is running for Nilotinib
Random Forest is running for Nutlin-3
Random Forest is running for Paclitaxel

Warning message in cor.test.default(train_Drug_data$IC50, train_prediction$prediction, :
"Cannot compute exact p-value with ties"


Random Forest is running for Panobinostat
Random Forest is running for PD-0325901
Random Forest is running for PD-0332991
Random Forest is running for PF2341066
Random Forest is running for PHA-665752
Random Forest is running for PLX4720
Random Forest is running for RAF265
Random Forest is running for Sorafenib
Random Forest is running for TAE684
Random Forest is running for TKI258
Random Forest is running for Topotecan
Random Forest is running for ZD-6474
Random Forest is running for mesothelioma
Random Forest is running for neuroblastoma
Random Forest is running for osteosarcoma
Random Forest is running for other
Random Forest is running for primitive_neuroectodermal_tumour-medulloblastoma
Random Forest is running for rhabdoid_tumour
Random Forest is running for rhabdomyosarcoma
Random Forest is running for sarcoma

In [ ]:
 ALL_RESULT_RF  <- ldply(outer_list, "rbind")
     All_RF_RESULT  <- All_RESULT_RF[order(All_RESULT_RF$R_Square_Test , decreasing = TRUE),]
     write.table(All_RF_RESULT,
            "C:/Users/MONSTER/Desktop/TEZ/CCLE/All_RF_RESULT.tsv",
            row.names = FALSE,
            quote = FALSE,
            sep = "\t"
            
           )